## Workshop - OLS Python

In this workshop, we are going to:

1. perform backward selection on the class data set
   1. fit the full model with $\%\Delta rGDP$ as the label
   2. remove the feature with the highest p-value
   3. refit the model
   4. repeat steps B. and C. until all features have p-values below 0.05
2. evaluatate the model performance

*Do not use interactions or polynomial terms in this workshop.*

# Preliminaries

- Load any necessary packages and/or functions
    * For backward select, I recommend using `statsmodels.api` instead of `statsmodels.formula.api`. Your choice.
- Load in the class data
- Define `x` and `y`
- Create a train-test split with
    * training size of two-thirds
    * random state of 490

In [1]:
import pandas as pd
import numpy as np

import statsmodels.api as sm

from sklearn.model_selection import train_test_split

In [2]:
# To load in the pickle
df = pd.read_pickle('C:/Users/johnj/Documents/Data/aml in econ 02 spring 2021/class data/class_data.pkl')

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/johnj/Documents/Data/aml in econ 02 spring 2021/class data/class_data.pkl'

In [3]:
# To load in and set up the csv file
df = pd.read_csv('C:/Users/johnj/Documents/Data/aml in econ 02 spring 2021/class data/class_data.csv')
df.set_index(['fips', 'year', 'GeoName'], inplace = True)
df
df['year'] = df.index.get_level_values('year')

In [4]:
# defining y and x
y = df['pct_d_rgdp']
x = df.drop(columns = 'pct_d_rgdp')

# Creating dummies
x = x.join([pd.get_dummies(x['year'], prefix = 'year', drop_first = True),
          pd.get_dummies(x['urate_bin'], prefix = 'urate', drop_first = True)]).drop(columns = ['year', 'urate_bin'])
x = sm.add_constant(x)

# Sorting the columns for output
x.sort_index(axis = 'columns', inplace = True)

# Dropping un
x.columns

Index(['const', 'density', 'emp_estabs', 'estabs_entry_rate',
       'estabs_exit_rate', 'lfpr', 'pop', 'pop_pct_black', 'pop_pct_hisp',
       'pos_net_jobs', 'urate_lower', 'urate_similar', 'year_2003',
       'year_2004', 'year_2005', 'year_2006', 'year_2007', 'year_2008',
       'year_2009', 'year_2010', 'year_2011', 'year_2012', 'year_2013',
       'year_2014', 'year_2015', 'year_2016', 'year_2017', 'year_2018'],
      dtype='object')

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 2/3,
                                                    random_state = 490)

*****
# Backward Selection 

In [6]:
fit = sm.OLS(y_train, x_train.drop(columns = ['density',
                                             'year_2005',
                                             'pop_pct_black',
                                             'year_2013',
                                             'year_2003',
                                             'pop'])).fit()
print(fit.summary2())

                  Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     0.041      
Dependent Variable: pct_d_rgdp       AIC:                246970.3167
Date:               2021-02-23 17:56 BIC:                247155.7953
No. Observations:   33889            Log-Likelihood:     -1.2346e+05
Df Model:           21               F-statistic:        69.38      
Df Residuals:       33867            Prob (F-statistic): 1.65e-289  
R-squared:          0.041            Scale:              85.550     
--------------------------------------------------------------------
                     Coef.  Std.Err.    t     P>|t|   [0.025  0.975]
--------------------------------------------------------------------
const               -2.1414   0.5048  -4.2422 0.0000 -3.1308 -1.1520
emp_estabs          -0.0434   0.0108  -4.0255 0.0001 -0.0645 -0.0223
estabs_entry_rate    0.2878   0.0197  14.6260 0.0000  0.2492  0.3263
estabs_exit_rate    -0.2135   0.0226  -9.4646 0.0000 

**********
# Testing

Evaluate two RMSEs:

1. null model
2. backward-selected model

Then, determine the percent improvement of the backward-selected modelf from the null model.

In [7]:
rmse_null = np.sqrt(  np.mean((y_test - np.mean(y_train))**2)  )
rmse_null

9.403229309446852

In [8]:
yhat_fit = fit.predict(x_test.drop(columns = ['density',
                                             'year_2005',
                                             'pop_pct_black',
                                             'year_2013',
                                             'year_2003',
                                             'pop']))
rmse_fit = np.sqrt(  np.mean((y_test - yhat_fit)**2)  )
rmse_fit

9.216942512937903

In [9]:
print(round((rmse_null - rmse_fit)/rmse_null*100, 2), '%', sep = '')

1.98%
